In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, r2_score
import os

# Carregar os dados do Google Drive
url = "https://drive.google.com/uc?id=1YcR0n1BECjwpihT-O93VNnkgtRhdLOBD"
data = pd.read_csv(url)

# Verifique se a coluna 'Minutos' existe após a leitura
if 'Minutos' not in data.columns:
    raise KeyError("A coluna 'Minutos' não foi encontrada no DataFrame.")

# Função para converter a coluna 'Minutos'
def convert_minutos(minuto_str):
    minuto_str = str(minuto_str).strip()  # Remover espaços em branco
    if 'a' in minuto_str:  # Se for um intervalo
        partes = minuto_str.split(' ao ')
        try:
            return (int(partes[0]) + int(partes[1])) / 2  # Média do intervalo
        except ValueError:
            return np.nan  # Retornar NaN para entradas não numéricas
    else:
        try:
            return float(minuto_str)  # Se for um número simples
        except ValueError:
            return np.nan  # Retornar NaN para entradas não numéricas

# Converter a coluna 'Minutos' para valores numéricos
data['Minutos'] = data['Minutos'].apply(convert_minutos)

# Remover linhas com valores NaN em 'Minutos'
data.dropna(subset=['Minutos'], inplace=True)

# Verifique novamente se a coluna 'Minutos' existe após a conversão
if 'Minutos' not in data.columns:
    raise KeyError("A coluna 'Minutos' não está disponível após a conversão.")

# Converter a coluna 'Resultado' em valores binários
data['Resultado'] = data['Resultado'].map({'Green': 1, 'Red': 0})

# Preparar os dados para o modelo
X = data[['Minutos', 'Odd', 'Investimento']]
y = data['Resultado']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)

# Calcular métricas de avaliação
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"R²: {r2:.2f}")

# Verifica se o arquivo de registros do usuário já existe
if os.path.exists('user_records.csv'):
    user_records = pd.read_csv('user_records.csv')
else:
    # DataFrame para armazenar os registros do usuário
    user_records = pd.DataFrame(columns=['Intervalo', 'Odd', 'Investimento', 'Recomendacao'])

# Função para buscar apostas semelhantes
def buscar_apostas_similares(intervalo, odd, investimento, resultado, tolerancia=0.25):
    # Filtrando apostas semelhantes
    similar_apostas = data[
        (data['Resultado'] == resultado) &
        (data['Odd'] >= odd * (1 - tolerancia)) &
        (data['Odd'] <= odd * (1 + tolerancia)) &
        (data['Investimento'] >= investimento * (1 - tolerancia)) &
        (data['Investimento'] <= investimento * (1 + tolerancia)) &
        (data['Minutos'] >= convert_minutos(intervalo) * (1 - tolerancia)) &
        (data['Minutos'] <= convert_minutos(intervalo) * (1 + tolerancia))
    ]

    # Mostrando os resultados
    if not similar_apostas.empty:
        print(f"\nResultados das apostas semelhantes que acabaram em {'Green' if resultado == 1 else 'Red'}:")
        for index, row in similar_apostas.iterrows():
            print(f"ID: {index}, Jogo: {row.get('Jogo', 'Informação não disponível')}, Minutos: {row['Minutos']}, Odd: {row['Odd']}, Investimento: {row['Investimento']}, Resultado: {'Green' if row['Resultado'] == 1 else 'Red'}")

# Exemplo de uso
while True:
    print("\nInsira as informações para previsão:")
    intervalo = input("Intervalo (Ex: '10 ao 20'): ")
    odd = float(input("Odd: "))
    investimento = float(input("Valor do investimento: "))

    # Fazer a previsão com base nas informações inseridas pelo usuário
    minutos = convert_minutos(intervalo)
    features = pd.DataFrame([[minutos, odd, investimento]], columns=['Minutos', 'Odd', 'Investimento'])
    recomendacao = model.predict(features)[0]

    # Resultado baseado na previsão
    if recomendacao == 1:
        print("Recomendação: Apostar!")
    else:
        print("Recomendação: Não apostar.")

    # Buscar apostas semelhantes
    buscar_apostas_similares(intervalo, odd, investimento, 1)  # Green
    buscar_apostas_similares(intervalo, odd, investimento, 0)  # Red

    # Armazenar o registro do usuário
    user_records = pd.concat([user_records, pd.DataFrame([{
        'Intervalo': intervalo,
        'Odd': odd,
        'Investimento': investimento,
        'Recomendacao': 'Apostar' if recomendacao == 1 else 'Não apostar'
    }])], ignore_index=True)

    continuar = input("\nDeseja inserir mais informações? (s/n): ")
    if continuar.lower() != 's':
        break

# Salvar os registros do usuário em um arquivo CSV
user_records.to_csv('user_records.csv', index=False)

# Exibir todos os registros do usuário
print("\nRegistros do usuário:")
print(user_records)


Acurácia do modelo: 0.98
Recall: 1.00
F1-Score: 0.99
R²: 0.74

Insira as informações para previsão:


<ipython-input-1-6e41f720d041>:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  user_records = pd.concat([user_records, pd.DataFrame([{


Recomendação: Apostar!

Resultados das apostas semelhantes que acabaram em Green:
ID: 50, Jogo: Aksur x Atyraur , Minutos: 15.0, Odd: 1.66, Investimento: 10, Resultado: Green
ID: 133, Jogo: Arges x Arad , Minutos: 15.0, Odd: 1.66, Investimento: 10, Resultado: Green
ID: 578, Jogo: Al Shamal x Al Gharafa, Minutos: 15.0, Odd: 1.66, Investimento: 10, Resultado: Green
ID: 616, Jogo: Mioveni x Arges, Minutos: 15.0, Odd: 1.66, Investimento: 10, Resultado: Green
ID: 707, Jogo: Al Shamal x Al Arab, Minutos: 15.0, Odd: 1.66, Investimento: 10, Resultado: Green
ID: 711, Jogo: Trabzonspor x Copenhagen, Minutos: 15.0, Odd: 2.28, Investimento: 10, Resultado: Green

Resultados das apostas semelhantes que acabaram em Red:
ID: 191, Jogo: Guilhermo x Chacarita , Minutos: 15.0, Odd: 1.66, Investimento: 10, Resultado: Red


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score, confusion_matrix

def avaliar_modelo(modelo, X_test, y_test):
    """
    Avalia o desempenho do modelo de aprendizado de máquina usando várias métricas.

    Parâmetros:
    modelo (objeto): Modelo treinado de aprendizado de máquina.
    X_test (DataFrame): Conjunto de dados de teste com recursos.
    y_test (Series): Valores reais do conjunto de dados de teste.

    Retorno:
    dict: Dicionário com as métricas de avaliação do modelo.
    """
    # Prever os resultados usando o conjunto de teste
    y_pred = modelo.predict(X_test)

    # Calcular as métricas
    acuracia = accuracy_score(y_test, y_pred)
    precisao = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    matriz_confusao = confusion_matrix(y_test, y_pred)

    # Exibir as métricas
    print(f"Acurácia: {acuracia:.2f}")
    print(f"Precisão: {precisao:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"R²: {r2:.2f}")
    print(f"Matriz de Confusão:\n{matriz_confusao}")

    # Retornar as métricas em um dicionário para possível uso posterior
    return {
        "Acurácia": acuracia,
        "Precisão": precisao,
        "Recall": recall,
        "F1-Score": f1,
        "R²": r2,
        "Matriz de Confusão": matriz_confusao
    }

# Exemplo de uso com o modelo e dados de teste
metricas = avaliar_modelo(model, X_test, y_test)


Acurácia: 0.98
Precisão: 0.97
Recall: 1.00
F1-Score: 0.99
R²: 0.74
Matriz de Confusão:
[[ 10   3]
 [  0 115]]
